In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import folium 

In [3]:
def load_distance_time_matrix(path, N):
    data= pd.read_csv(path)
    distance = np.zeros((N,N))
    time = np.zeros((N,N))
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen-1, destino-1] = float(data.iloc[i, 2])
        time[origen-1, destino-1] = float(data.iloc[i, 3])
    return distance, time

def load_vehicles(path):
    data= pd.read_csv(path)
    N = len(data)
    vehicles = {}
    for i in range(N):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    
    return vehicles

def load_demand(clientsPath):
    data= pd.read_csv(clientsPath)
    N = len(data)
    demand_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    
    return demand_dic


def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        capacity = float(data.iloc[i, 4])
        capacity_dic[id] = capacity
    
    return capacity_dic

def load_distance_time_dic(path, N):
    data= pd.read_csv(path)
    distance = {}
    time = {}
    
    for i in range(N):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    
    return distance, time

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot= pd.read_csv(depotsPath)
    client= pd.read_csv(clientsPath)
    
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]

    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    
    return coord
        
        



In [4]:
#Datos artificiales

np.random.seed(42)
n = 25  # 24 clientes + 1 depósito

dist_matrix_np, time_matrix = load_distance_time_matrix('..\Datos\Caso_Base\casoBase.csv', 25)
# Convertir a diccionario {(i,j): distancia}
distancia = {(i, j): dist_matrix_np[i][j]/1000 for i in range(n) for j in range(n)}

"""
vehiculos = {
  1: (130, 170),
  2: (140, 200),
  3: (120, 180),
  4: (100, 90),
  5: (70, 100),
  6: (55, 170),
  7: (110, 150),
  8: (114, 140)
}

demanda = {
  0:0, 1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15,
  11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12,
  19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11
}
"""

vehiculos = load_vehicles('../Datos/Caso_Base/vehicles.csv')
demanda = load_demand('..\Datos\Caso_Base\clients.csv')
coord = load_coordinates('..\Datos\Caso_Base\depots.csv', '..\Datos\Caso_Base\clients.csv')
#print(coord)


In [7]:

# Crear el modelo
model = ConcreteModel()

# Conjuntos e índices
nodos = [1] + list(demanda.keys())  # El nodo 1 es el depósito
model.N = Set(initialize=nodos)
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos
model.D = Set(initialize=[1])  # Nodo depósito

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i->j) por vehículo v
model.u = Var(model.N, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i por vehículo v

# Parámetros

# Asumiendo que tus nodos van del 1 al n
offset = 1
dist_modificada = {(i+offset, j+offset): v for (i, j), v in distancia.items()}
model.dist = Param(model.N, model.N, initialize=dist_modificada, default=0)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función Objetivo
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

# Cada cliente es visitado exactamente una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for i in model.N if i != c for v in model.V) == 1
    )

# Flujo conservado por vehículo en cada cliente
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for i in model.N if i != c) ==
            sum(model.x[c, j, v] for j in model.N if j != c)
        )

# Salida del depósito por vehículo es máximo 1
model.depot_departure = ConstraintList()
for v in model.V:
    model.depot_departure.add(
        sum(model.x[1, j, v] for j in model.N if j != 1) == 1  # El nodo depósito es 1
    )

# Capacidad del vehículo no debe excederse
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.cap[v]
    )

# Valor inicial de carga en el depósito es 1
model.initial_load = ConstraintList()
for v in model.V:
    model.initial_load.add(model.u[1, v] == 1)

# La carga nunca puede superar la capacidad del vehículo
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Rango del vehículo no debe excederse
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
    )

# No permitir ciclos triviales (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)

# Resolver el modelo
opt = SolverFactory('glpk')
results = opt.solve(model, tee=False)

# Resultados
print("Valor de la función objetivo:", model.obj())
# Mostrar capacidad utilizada por ruta
routes = {}
for v in model.V:
    print(f"Vehículo {v}:")
    routes[v] = []
    for i in model.N:
        for j in model.N:
            if i != j and model.x[i, j, v].value > 0.5:  # Solo si la ruta es tomada
                origen = coord[i]
                destino = coord[j]
                
                if origen not in routes[v]:
                    routes[v].append(origen)
                if destino not in routes[v]:
                    routes[v].append(destino)
                
                if i in model.C:
                    capacidad_utilizada = model.demand[i] * model.x[i, j, v].value
                    print(f"  Ruta {i} -> {j}: Carga transportada = {capacidad_utilizada}")
                else:
                    print(f"  Ruta {i} -> {j} (desde depósito)")



Valor de la función objetivo: 315.67189999999994
Vehículo 1:
  Ruta 1 -> 4 (desde depósito)
  Ruta 2 -> 16: Carga transportada = 13.0
  Ruta 4 -> 1: Carga transportada = 12.0
  Ruta 5 -> 2: Carga transportada = 15.0
  Ruta 10 -> 12: Carga transportada = 20.0
  Ruta 12 -> 10: Carga transportada = 17.0
  Ruta 16 -> 5: Carga transportada = 17.0
  Ruta 18 -> 25: Carga transportada = 25.0
  Ruta 25 -> 18: Carga transportada = 11.0
Vehículo 2:
  Ruta 1 -> 3 (desde depósito)
  Ruta 3 -> 1: Carga transportada = 15.0
  Ruta 8 -> 22: Carga transportada = 17.0
  Ruta 14 -> 23: Carga transportada = 21.0
  Ruta 22 -> 8: Carga transportada = 14.0
  Ruta 23 -> 14: Carga transportada = 18.0
Vehículo 3:
  Ruta 1 -> 21 (desde depósito)
  Ruta 21 -> 1: Carga transportada = 15.0
Vehículo 4:
  Ruta 1 -> 13 (desde depósito)
  Ruta 13 -> 1: Carga transportada = 12.0
Vehículo 5:
  Ruta 1 -> 24 (desde depósito)
  Ruta 24 -> 1: Carga transportada = 15.0
Vehículo 6:
  Ruta 1 -> 15 (desde depósito)
  Ruta 11 -> 1

## Visualización en mapa


In [ ]:
#print(routes[1])
m = folium.Map(
    location=[4.743359, -74.153536],
    zoom_start=12,
    tiles='OpenStreetMap' 
)


for route in routes.keys():
    if len(routes[route]) != 0 :  
        folium.PolyLine(
            routes[route],
            color='blue',
            weight=5,
            opacity=0.7,
            tooltip='Ruta A'
        ).add_to(m)

        folium.Marker(routes[route][0], popup="Inicio", icon=folium.Icon(color='blue')).add_to(m)

m

1
2
4
8
